In [1]:
import time
t1 = time.time()

import math
import os
import pandas as pd
import uproot
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import ripser
import persim

In [2]:
##### setting file path #####

sig_rootpath = "/data/Chen-Wang/gghh_highPT_250_200K/Events/run_01_decayed_1/tag_1_delphes_events.root"
sig_file = uproot.open(sig_rootpath)

ttbarBG_rootpath = "/data/Chen-Wang/ttbarBG_200K/Events/run_01/tag_1_delphes_events.root"
ttbarBG_file = uproot.open(ttbarBG_rootpath)

jjBG_rootpath = "/data/Chen-Wang/jjBG_200PT_200K/Events/run_01/tag_1_delphes_events.root"
jjBG_file = uproot.open(jjBG_rootpath)

In [3]:
##### include signal data #####

sig_events = [sig_file["Delphes;1"]["FatJet"].array(),
        sig_file["Delphes;1"]["FatJet.Eta"].array(),
        sig_file["Delphes;1"]["FatJet.Phi"].array(),
        sig_file["Delphes;1"]["FatJet.PT"].array(),
        sig_file["Delphes;1"]["FatJet.Mass"].array(),
        sig_file["Delphes;1"]["FatJet.Tau[5]"].array(),
        sig_file["Delphes;1"]["FatJet.Particles"].array(),
        sig_file["Delphes;1"]["Particle.PT"].array(),
        sig_file["Delphes;1"]["Particle.Eta"].array(),
        sig_file["Delphes;1"]["Particle.Phi"].array()
        ]

##### reshape the signal data #####

sig_events = np.expand_dims(sig_events, axis=-1)
sig_events = sig_events.transpose((1,0,2))
sig_events = np.squeeze(sig_events,axis=(2,))

num_sig = len(sig_events)

##### include ttbarBG data #####

ttbarBG_events = [ttbarBG_file["Delphes;1"]["FatJet"].array(),
        ttbarBG_file["Delphes;1"]["FatJet.Eta"].array(),
        ttbarBG_file["Delphes;1"]["FatJet.Phi"].array(),
        ttbarBG_file["Delphes;1"]["FatJet.PT"].array(),
        ttbarBG_file["Delphes;1"]["FatJet.Mass"].array(),
        ttbarBG_file["Delphes;1"]["FatJet.Tau[5]"].array(),
        ttbarBG_file["Delphes;1"]["FatJet.Particles"].array(),
        ttbarBG_file["Delphes;1"]["Particle.PT"].array(),
        ttbarBG_file["Delphes;1"]["Particle.Eta"].array(),
        ttbarBG_file["Delphes;1"]["Particle.Phi"].array()
        ]

##### reshape the ttbarBG data #####

ttbarBG_events = np.expand_dims(ttbarBG_events, axis=-1)
ttbarBG_events = ttbarBG_events.transpose((1,0,2))
ttbarBG_events = np.squeeze(ttbarBG_events,axis=(2,))

num_ttbarBG = len(ttbarBG_events)

##### include jjBG data #####

jjBG_events = [jjBG_file["Delphes;1"]["FatJet"].array(),
        jjBG_file["Delphes;1"]["FatJet.Eta"].array(),
        jjBG_file["Delphes;1"]["FatJet.Phi"].array(),
        jjBG_file["Delphes;1"]["FatJet.PT"].array(),
        jjBG_file["Delphes;1"]["FatJet.Mass"].array(),
        jjBG_file["Delphes;1"]["FatJet.Tau[5]"].array(),
        jjBG_file["Delphes;1"]["FatJet.Particles"].array(),
        jjBG_file["Delphes;1"]["Particle.PT"].array(),
        jjBG_file["Delphes;1"]["Particle.Eta"].array(),
        jjBG_file["Delphes;1"]["Particle.Phi"].array()
        ]

##### reshape the jjBG data #####

jjBG_events = np.expand_dims(jjBG_events, axis=-1)
jjBG_events = jjBG_events.transpose((1,0,2))
jjBG_events = np.squeeze(jjBG_events,axis=(2,))

num_jjBG = len(jjBG_events)

/usr/local/lib/python3.8/dist-packages/awkward/array/base.py:394: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return cls.numpy.array(value, copy=False)


In [4]:
##### useful function #####

##### select if Fat Jet >= 2 #####

def Fat_Jet_selection(events):
    where1 = np.where(events[:,0]>=2)    ### 0: number of fat jet
    return events[where1]

##### select if M_jet > 50 GeV #####

def mass_selection(events):
    where1 = []
    for i in range(len(events)):
        switch=1
        if events[i][4][0]<50:
            switch=0
        if events[i][4][1]<50:
            switch=0
        if switch==1:
            where1.append(i)
    return events[where1]

##### calculate X_HH #####

def cal_X_HH(jet_mass1, jet_mass2):
    diff1 = jet_mass1 - 124
    diff2 = jet_mass2 - 124
    if diff1<diff2:
        m1 = jet_mass1
        m2 = jet_mass2
    else:
        m1 = jet_mass2
        m2 = jet_mass1
    return np.sqrt(((m1-124)/(0.1*m1+0.00001))**2 + ((m2-115)/(0.1*m2+0.00001))**2)

##### calculate iht #####

def cal_iht(events):
    return events[3][0]+events[3][1]

##### calculate M_JJ #####

def cal_m_JJ(events):
    p = [0,0,0,0]    ### four momentum
    for i in range(2):    ### leading jet and sub-leading jet
        pt = events[3][i]    ### 3: pT of fat jet
        eta = events[1][i]    ### 1: eta of fat jet
        phi = events[2][i]    ### 2: phi of fat jet
        mass = events[4][i]    ### 4: mass of fat jet
        p[1] = p[1] + pt*np.cos(phi)    ### px
        p[2] = p[2] + pt*np.sin(phi)    ### py
        p[3] = p[3] + pt*np.sinh(eta)    ### pz
        p[0] = p[0] + np.sqrt( mass**2 + (pt*np.cos(phi))**2 + (pt*np.sin(phi))**2 + (pt*np.sinh(eta))**2 )    ### energy
    m_JJ = np.sqrt(p[0]**2 - p[1]**2 - p[2]**2 - p[3]**2)
    return m_JJ

##### calculate deta_JJ #####

def cal_deta_JJ(events):
    eta_J1 = events[1][0]
    eta_J2 = events[1][1]
    return abs(eta_J1-eta_J2)

##### calculate J1_tau21 #####

def cal_J1_tau21(events):
    tau2 = events[5][0][1]
    tau1 = events[5][0][0]
    return tau2/tau1

##### calculate J2_tau21 #####

def cal_J2_tau21(events):
    tau2 = events[5][1][1]
    tau1 = events[5][1][0]
    return tau2/tau1

##### calculate TDA #####

def cal_TDA(event):
    where_jet_particle = event[6][0]+event[6][1]    ### 6: fat jet particles
    where_jet_particle = np.array(where_jet_particle)-1
    center_eta = (event[1][0]+event[1][1])/2    ### 1: fat jet eta
    center_phi = (event[2][0]+event[2][1])/2    ### 2: fat jet phi
    particle_eta = event[8][where_jet_particle]-center_eta    ### 8: particle eta, and centerize
    particle_phi = event[9][where_jet_particle]-center_phi    ### 9: particle phi, and centerize

    where_larger_pi = np.where(particle_phi>np.pi)
    particle_phi[where_larger_pi] = -(2*np.pi - particle_phi[where_larger_pi])    ### check if jet particle split
    where_smaller_mpi = np.where(particle_phi<-np.pi)
    particle_phi[where_smaller_mpi] = 2*np.pi + particle_phi[where_smaller_mpi]    ### check if jet particle split
    P = np.array([particle_eta, particle_phi]).T
    diagrams = ripser.ripser(P)['dgms']
    return diagrams

##### calculate TDA sum of lifetime #####

def cal_TDA_sum(diagrams):
    sum_l0 = np.sum(diagrams[0][:-1,1] - diagrams[0][:-1,0])
    sum_l1 = np.sum(diagrams[1][:,1] - diagrams[1][:,0])
    return sum_l0, sum_l1

##### calculate TDA mean of lifetime #####

def cal_TDA_mean(diagrams):
    l0 = diagrams[0][:-1,1]-diagrams[0][:-1,0]
    l1 = diagrams[1][:,1]-diagrams[1][:,0]
    if np.sum(l1)==0:
        l1 = np.concatenate([l1,[0]])
    return np.mean(l0), np.mean(l1)

##### calculate TDA entropy #####

def cal_TDA_entropy(diagrams):
    l0 = diagrams[0][:-1,1]-diagrams[0][:-1,0]
    L0 = np.sum(l0)
    S0 = (l0/L0)*np.log2((l0/L0))
    l1 = diagrams[1][:,1]-diagrams[1][:,0]
    L1 = np.sum(l1)
    S1 = (l1/L1)*np.log2((l1/L1))
    return -np.sum(S0), -np.sum(S1)

##### calculate TDA Lifetime*Birthtime #####

def cal_TDA_LB(diagrams):
    return np.sum(diagrams[1][:,0]*(diagrams[1][:,1]-diagrams[1][:,0]))

In [5]:
##### basic selection #####

sig_events = Fat_Jet_selection(sig_events)
sig_events = mass_selection(sig_events)
print("There are", len(sig_events), "signal events left after the selection with originally event number", num_sig)
num_sig = len(sig_events)

ttbarBG_events = Fat_Jet_selection(ttbarBG_events)
ttbarBG_events = mass_selection(ttbarBG_events)
print("There are", len(ttbarBG_events), "ttbarBG events left after the selection with originally event number", num_ttbarBG)
num_ttbarBG = len(ttbarBG_events)

jjBG_events = Fat_Jet_selection(jjBG_events)
jjBG_events = mass_selection(jjBG_events)
print("There are", len(jjBG_events), "jjBG events left after the selection with originally event number", num_jjBG)
num_jjBG = len(jjBG_events)

There are 140271 signal events left after the selection with originally event number 200000
There are 109522 ttbarBG events left after the selection with originally event number 200000
There are 54308 jjBG events left after the selection with originally event number 200000


In [6]:
##### calculate high level featueres #####

X_HH = []
for i in range(len(sig_events)):
    X_HH.append(cal_X_HH(sig_events[i][4][0], sig_events[i][4][1]))
for i in range(len(ttbarBG_events)):
    X_HH.append(cal_X_HH(ttbarBG_events[i][4][0], ttbarBG_events[i][4][1]))
for i in range(len(jjBG_events)):
    X_HH.append(cal_X_HH(jjBG_events[i][4][0], jjBG_events[i][4][1]))
print("X_HH finished after time:", time.time()-t1)

iht = []
for i in range(len(sig_events)):
    iht.append(cal_iht(sig_events[i]))
for i in range(len(ttbarBG_events)):
    iht.append(cal_iht(ttbarBG_events[i]))
for i in range(len(jjBG_events)):
    iht.append(cal_iht(jjBG_events[i]))
print("iht finished after time:", time.time()-t1)

m_JJ = []
for i in range(len(sig_events)):
    m_JJ.append(cal_m_JJ(sig_events[i]))
for i in range(len(ttbarBG_events)):
    m_JJ.append(cal_m_JJ(ttbarBG_events[i]))
for i in range(len(jjBG_events)):
    m_JJ.append(cal_m_JJ(jjBG_events[i]))
print("m_JJ finished after time:", time.time()-t1)

m_J1 = []
for i in range(len(sig_events)):
    m_J1.append(sig_events[i][4][0])
for i in range(len(ttbarBG_events)):
    m_J1.append(ttbarBG_events[i][4][0])
for i in range(len(jjBG_events)):
    m_J1.append(jjBG_events[i][4][0])
print("m_J1 finished after time:", time.time()-t1)

m_J2 = []
for i in range(len(sig_events)):
    m_J2.append(sig_events[i][4][1])
for i in range(len(ttbarBG_events)):
    m_J2.append(ttbarBG_events[i][4][1])
for i in range(len(jjBG_events)):
    m_J2.append(jjBG_events[i][4][1])
print("m_J2 finished after time:", time.time()-t1)

deta_JJ = []
for i in range(len(sig_events)):
    deta_JJ.append(cal_deta_JJ(sig_events[i]))
for i in range(len(ttbarBG_events)):
    deta_JJ.append(cal_deta_JJ(ttbarBG_events[i]))
for i in range(len(jjBG_events)):
    deta_JJ.append(cal_deta_JJ(jjBG_events[i]))
print("deta_JJ finished after time:", time.time()-t1)

J1_tau21 = []
for i in range(len(sig_events)):
    J1_tau21.append(cal_J1_tau21(sig_events[i]))
for i in range(len(ttbarBG_events)):
    J1_tau21.append(cal_J1_tau21(ttbarBG_events[i]))
for i in range(len(jjBG_events)):
    J1_tau21.append(cal_J1_tau21(jjBG_events[i]))
print("J1_tau21 finished after time:", time.time()-t1)

J2_tau21 = []
for i in range(len(sig_events)):
    J2_tau21.append(cal_J2_tau21(sig_events[i]))
for i in range(len(ttbarBG_events)):
    J2_tau21.append(cal_J2_tau21(ttbarBG_events[i]))
for i in range(len(jjBG_events)):
    J2_tau21.append(cal_J2_tau21(jjBG_events[i]))
print("J2_tau21 finished after time:", time.time()-t1)

X_HH finished after time: 238.83114314079285
iht finished after time: 239.02436709403992
m_JJ finished after time: 251.97892808914185
m_J1 finished after time: 252.09609460830688
m_J2 finished after time: 252.2039020061493
deta_JJ finished after time: 252.42105174064636
J1_tau21 finished after time: 252.7119734287262
J2_tau21 finished after time: 252.98812317848206


In [7]:
##### calculate TDA featueres #####

H_0 = []
H_1 = []
SS_0 = []
SS_1 = []
LB_1 = []

for i in range(len(sig_events)):
    TDA_tmp = cal_TDA(sig_events[i])
    H_0.append(cal_TDA_sum(TDA_tmp)[0])        ### H is the sum of lifetime
    H_1.append(cal_TDA_sum(TDA_tmp)[1])
    SS_0.append(cal_TDA_entropy(TDA_tmp)[0])   ### SS is the entropy of lifetime
    SS_1.append(cal_TDA_entropy(TDA_tmp)[1])
    LB_1.append(cal_TDA_LB(TDA_tmp))           ### LB is the lifetime*birthtime
print("signal TDA features finished after time:", time.time()-t1)
    
for i in range(len(ttbarBG_events)):
    TDA_tmp = cal_TDA(ttbarBG_events[i])
    H_0.append(cal_TDA_sum(TDA_tmp)[0])
    H_1.append(cal_TDA_sum(TDA_tmp)[1])
    SS_0.append(cal_TDA_entropy(TDA_tmp)[0])
    SS_1.append(cal_TDA_entropy(TDA_tmp)[1])
    LB_1.append(cal_TDA_LB(TDA_tmp))
print("ttbarBG TDA features finished after time:", time.time()-t1)
    
for i in range(len(jjBG_events)):
    TDA_tmp = cal_TDA(jjBG_events[i])
    H_0.append(cal_TDA_sum(TDA_tmp)[0])
    H_1.append(cal_TDA_sum(TDA_tmp)[1])
    SS_0.append(cal_TDA_entropy(TDA_tmp)[0])
    SS_1.append(cal_TDA_entropy(TDA_tmp)[1])
    LB_1.append(cal_TDA_LB(TDA_tmp))
print("jjBG TDA features finished after time:", time.time()-t1)

signal TDA features finished after time: 912.38805103302
ttbarBG TDA features finished after time: 1392.0487961769104
jjBG TDA features finished after time: 1635.0381817817688


In [8]:
##### create true label #####

target2 = np.zeros(num_sig+num_ttbarBG+num_jjBG)
target2[num_sig:] = 1     ### signal label as 0

target3 = np.zeros(num_sig+num_ttbarBG+num_jjBG)
target3[0:num_sig] = 0     ### signal label as 0
target3[num_sig:num_sig+num_ttbarBG] = 1     ### ttbarBG label as 1
target3[num_sig+num_ttbarBG:] = 2     ### jjBG label as 2

In [9]:
d = {'X_HH':X_HH, 'iht':iht, 'm_JJ':m_JJ, 'm_J1':m_J1, 'm_J2':m_J2, 'deta_JJ':deta_JJ, 'J1_tau21':J1_tau21, 'J2_tau21':J2_tau21, 'H_0':H_0, 'H_1':H_1, 'SS_0':SS_0, 'SS_1':SS_1, 'LB_1':LB_1, 'target2':target2, 'target3':target3}
df = pd.DataFrame(data=d)
df

,X_HH,iht,m_JJ,m_J1,m_J2,deta_JJ,J1_tau21,J2_tau21,H_0,H_1,SS_0,SS_1,LB_1,target2,target3
0,2.169814,498.863251,552.356994,102.008881,112.236855,0.482302,0.221553,0.385671,13.233662,0.314678,5.709411,1.573460,0.355070,0.0,0.0
1,9.475008,593.795044,597.207660,67.299019,80.223694,0.341132,0.742909,0.179675,10.209827,0.090669,4.825404,2.549878,0.018734,0.0,0.0
2,11.972177,505.011780,638.938097,56.588074,102.755157,1.329138,0.750892,0.413120,11.556087,0.207381,5.250363,2.171730,0.081837,0.0,0.0
3,3.305570,639.797974,709.878242,136.601318,168.479492,0.386040,0.395503,0.343482,14.189290,0.422620,6.027356,2.983404,0.161782,0.0,0.0
4,6.146391,496.671631,524.943021,88.915337,80.513718,0.158647,0.406887,0.329383,8.284440,0.056204,5.248461,2.160549,0.004379,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304096,10.294289,475.100525,1057.486071,91.752502,62.069443,2.825930,0.536810,0.546817,10.345854,0.091178,4.800460,1.766998,0.021480,1.0,2.0
304097,6.368375,717.999756,834.096856,150.703796,77.932030,1.255619,0.667381,0.633477,14.880385,0.532661,6.254151,3.535876,0.111021,1.0,2.0
304098,14.665185,541.429565,1300.913720,81.542030,51.496067,3.036777,0.637304,0.842317,13.297320,0.096435,5.286486,2.383225,0.015745,1.0,2.0
304099,4.981194,608.272217,810.263658,88.551506,88.705063,1.538944,0.437125,0.545170,12.983457,0.201125,5.350721,2.235349,0.070455,1.0,2.0


In [10]:
##### output test data #####
import h5py

h5f = h5py.File('/data/Chen-Wang/test_data_HLfeature.h5', 'w')
h5f.create_dataset('X_HH', data=X_HH)
h5f.create_dataset('iht', data=iht)
h5f.create_dataset('m_JJ', data=m_JJ)
h5f.create_dataset('m_J1', data=m_J1)
h5f.create_dataset('m_J2', data=m_J2)
h5f.create_dataset('deta_JJ', data=deta_JJ)
h5f.create_dataset('J1_tau21', data=J1_tau21)
h5f.create_dataset('J2_tau21', data=J2_tau21)
h5f.create_dataset('H_0', data=H_0)
h5f.create_dataset('H_1', data=H_1)
h5f.create_dataset('SS_0', data=SS_0)
h5f.create_dataset('SS_1', data=SS_1)
h5f.create_dataset('LB_1', data=LB_1)
h5f.create_dataset('target2', data=target2)
h5f.create_dataset('target3', data=target3)
h5f.close()